In [ ]:
import numpy as np
from skyfield.framelib import itrs
from coverage import *

import geopandas as gpd
import pandas as pd
import shapely
from shapely.geometry import Polygon
import branca
import folium


In [ ]:
num_days = 3
tles = gen_sats(sat_nos=[39084,49260])

times = gen_times(start_yr=2021, start_mo=12, start_day=10, days=num_days, step_min=1)

# Generate the instrument from basic specs
# inst = gen_instrument(name="tirs", fl=178, pitch=0.025, h_pix=1850, v_pix=1800, mm=True)
inst = gen_instrument(name="tirs", fl=178, pitch=0.025, h_pix=1850, v_pix=4000, mm=True)

In [ ]:
# %%timeit

# get_inst_fov(tles[0][0], times[0], inst)

In [ ]:
gdfs = []
for tle in tles:
    sat = tle[0]

    def gen_fov_poly(time):
        # Get the ITRS position of the satellite as origin of LVLH frame.
        xyz_dist_rates = sat.at(time).frame_xyz_and_velocity(itrs)
        # xyz_dist = xyz_dist_rates[0]
        z_rate = xyz_dist_rates[1]
        
        # Descending only filter:
        if z_rate.km_per_s[2] < 0:
            cs_lla_dict = get_inst_fov(sat, time, inst)

            # Add lat, lon offset for each corner of FOV
            return Polygon([(cs_lla_dict["c1"]["lon"], cs_lla_dict["c1"]["lat"]), 
                    (cs_lla_dict["c2"]["lon"], cs_lla_dict["c2"]["lat"]), 
                    (cs_lla_dict["c3"]["lon"], cs_lla_dict["c3"]["lat"]),
                    (cs_lla_dict["c4"]["lon"], cs_lla_dict["c4"]["lat"]),
                    (cs_lla_dict["c1"]["lon"], cs_lla_dict["c1"]["lat"])]
                    )

    vfunc = np.vectorize(gen_fov_poly)
    polys = vfunc(times)

    poly_df = gpd.GeoDataFrame(
            data=polys, 
            columns=['geometry'], 
            crs="EPSG:4326"
            )
    poly_df["satellite"] = sat.name
    poly_df["id"] = np.abs(sat.target)
    poly_df["time"] = times.utc_strftime()

    gdfs.append(poly_df)

poly_df = gpd.GeoDataFrame(pd.concat(gdfs, ignore_index=True), crs="epsg:4326")
poly_df["lonspan"] = poly_df.bounds['maxx'] - poly_df.bounds['minx']

# Create cmap for unique satellites and create color column
sat_ids = list(poly_df["id"].unique()).sort()
cmap = branca.colormap.StepColormap(['red', 'blue'], sat_ids, vmin=139084, vmax = 149260)
poly_df['color'] = poly_df['id'].apply(cmap)


In [ ]:
## Select AOI from gpd naturalearth dataset (filter by .name for country, .continent for continent)
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))

# aoi =  world[world.name == "Brazil"].geometry
# aoi =  world[world.continent == "Africa"].geometry
aoi = world[world.name == "United States of America"].geometry

# Filter shapes crossing anti-meridian
plot_df = poly_df[poly_df["lonspan"] < 20].copy()

# Filter by AOI
xmin, ymin, xmax, ymax= aoi.total_bounds
plot_df = plot_df.cx[xmin: xmax, ymin: ymax]

In [ ]:
if num_days <= 16:
    m = plot_df.explore(color="color", tooltip=["satellite", "time"])
    display(m)
    #m.save("fov_output.html")

In [ ]:
## Coverage data analysis
# 1) Create grid of points with regular spacing in degrees
xcell_size = ycell_size = 0.3
# projection of the grid
crs = 'EPSG:4326'

xcells = np.arange(xmin, xmax+xcell_size, xcell_size)
ycells = np.arange(ymin, ymax+ycell_size, ycell_size)

# create the grid points in a loop
grid_points = []
for x0 in xcells:
    for y0 in ycells:
        grid_points.append(shapely.geometry.Point(x0, y0))

grid = gpd.GeoDataFrame(grid_points, columns=['geometry'], 
                                 crs=crs)


# 2) Add "n_visits" column to grid using sjoin/ dissolve
shapes = gpd.GeoDataFrame(plot_df.geometry)
merged = gpd.sjoin(shapes, grid, how='left', predicate="intersects")
merged['n_visits']=0 # this will be replaced with nan or positive int where n_visits > 0
dissolve = merged.dissolve(by="index_right", aggfunc="count") # no difference in count vs. sum here?
grid.loc[dissolve.index, 'n_visits'] = dissolve.n_visits.values

In [ ]:
grid.n_visits.describe()

In [ ]:
# Form 2D array of n_visits based on grid shape
img = np.rot90(grid.n_visits.values.reshape(len(xcells),len(ycells)))

# Create colormap and apply to img
colormap = branca.colormap.step.viridis.scale(1, grid.n_visits.max())

def colorfunc(x):
    if np.isnan(x):
        return (0,0,0,0)
    else:
        return colormap.rgba_bytes_tuple(x)

# Apply cmap to img array and rearrange for RGBA
cmap = np.vectorize(colorfunc)
rgba_img = np.array(cmap(img))
rgba_img = np.moveaxis(rgba_img, 0, 2)

# Update image corner bounds based on cell size
xmin, ymin, xmax, ymax= grid.total_bounds
xmin = xmin - xcell_size/2
ymin = ymin - ycell_size/2
xmax = xmax + xcell_size/2
ymax = ymax + ycell_size/2


In [ ]:
m = folium.Map()
m.fit_bounds([[ymin, xmin], [ymax, xmax]])
m.add_child(folium.raster_layers.ImageOverlay(rgba_img, opacity=.6, mercator_project=True,# crs="EPSG:4326",
                                 bounds = [[ymin,xmin],[ymax,xmax]]))
colormap.add_to(m)
m#.save("coverage_output.html")